# Exploration de la base de données vélib fournie par Depersin et Schreuder

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from json import loads #pour lire des dicos dans des strings

In [2]:
path = "velib_358.csv"
velib = pd.read_csv(path, sep = ';')

In [3]:
def to_dictionary(jour):
    """transformer le format chelou en dico"""

    dia = '{' + jour[2:-2].replace('{','').replace('}','').replace("'",'"') + '}'
    try :
        dictionary = loads(dia)
    except:
        dictionary = {'mechanical': 0, 'ebike': 0}
    
    return dictionary

col = velib.columns

In [4]:
velib = velib.set_index("station_id")

In [5]:
velib.head()

,10/19/20;18:01:51,10/19/20;18:06:52,10/19/20;18:11:52,10/19/20;18:16:52,10/19/20;18:21:52,10/19/20;18:26:53,10/19/20;18:31:53,10/19/20;18:36:53,10/19/20;18:41:54,10/19/20;18:46:54,...,11/03/20;16:01:46,11/03/20;16:06:47,11/03/20;16:11:47,11/03/20;16:16:47,11/03/20;16:21:48,11/03/20;16:26:48,11/03/20;16:31:48,11/03/20;16:36:49,11/03/20;16:41:49,11/03/20;16:46:49
station_id,,,,,,,,,,,,,,,,,,,,,
213688169,"[{'mechanical': 0}, {'ebike': 2}]","[{'mechanical': 0}, {'ebike': 2}]","[{'mechanical': 0}, {'ebike': 2}]","[{'mechanical': 0}, {'ebike': 3}]","[{'mechanical': 0}, {'ebike': 3}]","[{'mechanical': 1}, {'ebike': 2}]","[{'mechanical': 0}, {'ebike': 1}]","[{'mechanical': 0}, {'ebike': 1}]","[{'mechanical': 0}, {'ebike': 1}]","[{'mechanical': 0}, {'ebike': 2}]",...,"[{'mechanical': 4}, {'ebike': 6}]","[{'mechanical': 4}, {'ebike': 6}]","[{'mechanical': 3}, {'ebike': 6}]","[{'mechanical': 4}, {'ebike': 7}]","[{'mechanical': 5}, {'ebike': 7}]","[{'mechanical': 5}, {'ebike': 7}]","[{'mechanical': 5}, {'ebike': 8}]","[{'mechanical': 5}, {'ebike': 8}]","[{'mechanical': 5}, {'ebike': 8}]","[{'mechanical': 5}, {'ebike': 7}]"
99950133,"[{'mechanical': 41}, {'ebike': 5}]","[{'mechanical': 41}, {'ebike': 5}]","[{'mechanical': 42}, {'ebike': 5}]","[{'mechanical': 42}, {'ebike': 7}]","[{'mechanical': 41}, {'ebike': 5}]","[{'mechanical': 40}, {'ebike': 4}]","[{'mechanical': 38}, {'ebike': 5}]","[{'mechanical': 39}, {'ebike': 4}]","[{'mechanical': 40}, {'ebike': 5}]","[{'mechanical': 37}, {'ebike': 6}]",...,"[{'mechanical': 2}, {'ebike': 0}]","[{'mechanical': 2}, {'ebike': 0}]","[{'mechanical': 2}, {'ebike': 0}]","[{'mechanical': 2}, {'ebike': 1}]","[{'mechanical': 2}, {'ebike': 1}]","[{'mechanical': 2}, {'ebike': 0}]","[{'mechanical': 2}, {'ebike': 0}]","[{'mechanical': 2}, {'ebike': 0}]","[{'mechanical': 2}, {'ebike': 0}]","[{'mechanical': 2}, {'ebike': 0}]"
516709288,"[{'mechanical': 0}, {'ebike': 1}]","[{'mechanical': 0}, {'ebike': 1}]","[{'mechanical': 0}, {'ebike': 1}]","[{'mechanical': 1}, {'ebike': 1}]","[{'mechanical': 1}, {'ebike': 1}]","[{'mechanical': 1}, {'ebike': 1}]","[{'mechanical': 2}, {'ebike': 1}]","[{'mechanical': 0}, {'ebike': 1}]","[{'mechanical': 0}, {'ebike': 1}]","[{'mechanical': 0}, {'ebike': 1}]",...,"[{'mechanical': 1}, {'ebike': 2}]","[{'mechanical': 1}, {'ebike': 2}]","[{'mechanical': 1}, {'ebike': 2}]","[{'mechanical': 1}, {'ebike': 2}]","[{'mechanical': 1}, {'ebike': 2}]","[{'mechanical': 1}, {'ebike': 3}]","[{'mechanical': 1}, {'ebike': 2}]","[{'mechanical': 1}, {'ebike': 3}]","[{'mechanical': 1}, {'ebike': 3}]","[{'mechanical': 1}, {'ebike': 3}]"
36255,"[{'mechanical': 0}, {'ebike': 0}]","[{'mechanical': 0}, {'ebike': 0}]","[{'mechanical': 0}, {'ebike': 0}]","[{'mechanical': 0}, {'ebike': 0}]","[{'mechanical': 0}, {'ebike': 2}]","[{'mechanical': 0}, {'ebike': 4}]","[{'mechanical': 0}, {'ebike': 4}]","[{'mechanical': 0}, {'ebike': 4}]","[{'mechanical': 1}, {'ebike': 3}]","[{'mechanical': 2}, {'ebike': 2}]",...,"[{'mechanical': 0}, {'ebike': 3}]","[{'mechanical': 0}, {'ebike': 3}]","[{'mechanical': 0}, {'ebike': 3}]","[{'mechanical': 0}, {'ebike': 3}]","[{'mechanical': 0}, {'ebike': 2}]","[{'mechanical': 0}, {'ebike': 2}]","[{'mechanical': 0}, {'ebike': 3}]","[{'mechanical': 0}, {'ebike': 3}]","[{'mechanical': 0}, {'ebike': 3}]","[{'mechanical': 0}, {'ebike': 2}]"
37815204,"[{'mechanical': 4}, {'ebike': 1}]","[{'mechanical': 3}, {'ebike': 1}]","[{'mechanical': 1}, {'ebike': 1}]","[{'mechanical': 1}, {'ebike': 1}]","[{'mechanical': 2}, {'ebike': 1}]","[{'mechanical': 3}, {'ebike': 1}]","[{'mechanical': 3}, {'ebike': 2}]","[{'mechanical': 2}, {'ebike': 0}]","[{'mechanical': 2}, {'ebike': 0}]","[{'mechanical': 4}, {'ebike': 0}]",...,"[{'mechanical': 6}, {'ebike': 0}]","[{'mechanical': 7}, {'ebike': 0}]","[{'mechanical': 7}, {'ebike': 0}]","[{'mechanical': 7}, {'ebike': 1}]","[{'mechanical': 7}, {'ebike': 1}]","[{'mechanical': 8}, {'ebike': 1}]","[{'mechanical': 9}, {'ebike': 0}]","[{'mechanical': 10}, {'ebike': 0}]",

# Pour connaître le nom du projet et son fuseau horaire : table `system_formation` avec :
- language : langue utilisée dans l’ensemble des fichiers
- name : nom du système
- system_id : identifiant unique du système
- timezone : zone horaire associée au système
- url :adresse associée au système
- lastUpdatedOther : date de dernière mis-à jour des informations de la ville
- ttl : « time to live » , durée de vie de l’information au-delà de laquelle elle doit être considérée comme obsolète


In [17]:
import urllib.request
import json
from pandas.io.json import json_normalize

request= urllib.request.Request('https://velib-metropole-opendata.smoove.pro/opendata/Velib_Metropole/system_information.json')
response = urllib.request.urlopen(request)
station_info = response.read()
data = json.loads(station_info)
df_station_info = json_normalize(data['data'])
df_station_info.head()

,system_id,name,url,timezone,language
0,Paris,Paris,,Europe/Paris,en


# Pour connaître les flux existants : la table `gbfs`



In [12]:
request= urllib.request.Request('https://velib-metropole-opendata.smoove.pro/opendata/Velib_Metropole/gbfs.json')
response = urllib.request.urlopen(request)
flux_existants = response.read()
data = json.loads(flux_existants)
df_flux = json_normalize(data['data']['en']['feeds'])
df_flux

,name,url
0,gbfs,https://velib-metropole-opendata.smoove.pro/op...
1,system_information,https://velib-metropole-opendata.smoove.pro/op...
2,station_information,https://velib-metropole-opendata.smoove.pro/op...
3,station_status,https://velib-metropole-opendata.smoove.pro/op...


# Information sur les stations avec la table `station_information` : 
- capacity : nombre de bornettes dans la station
- lat : latitude de la station (au format WGS84)
- lon : longitude de la station (au format WGS84)
- name : nom de la station
- station_id : identifiant unique associé à la station. Ce numéro identifie la station au sein du service Vélib’ Métropole

In [13]:
request= urllib.request.Request('https://velib-metropole-opendata.smoove.pro/opendata/Velib_Metropole/station_information.json')
response = urllib.request.urlopen(request)
station_info = response.read()
data = json.loads(station_info)
df_station_info = json_normalize(data['data']['stations'])
df_station_info.head()

,station_id,name,lat,lon,capacity,stationCode,rental_methods
0,213688169,Benjamin Godard - Victor Hugo,48.865983,2.275725,35,16107,NaN
1,99950133,André Mazet - Saint-André des Arts,48.853756,2.339096,55,6015,[CREDITCARD]
2,516709288,Charonne - Robert et Sonia Delauney,48.855908,2.392571,20,11104,NaN
3,36255,Toudouze - Clauzel,48.879296,2.337360,21,9020,[CREDITCARD]
4,37815204,Mairie du 12ème,48.840855,2.387555,30,12109,NaN


# Nombre de vélos et de bornettes disponibles par station avec la table `stations_status` avec :
- station_id: numéro unique d’identification de la station. Ce numéro identifie la station au sein du service Vélib’ Métropole
- is_installed: variable binaire indiquant si la station est. La station a déjà été déployée (1) ou est encore en cours de déploiement (0) 
- is_renting: variable binaire indiquant si la station peut louer des vélos (is_renting=1 si le statut de la station est Operative)
- is_returning: variable binaire indiquant si la station peut recevoir des vélos (is_renting=1 si le statut de la station est Operative)
- last_reported : date de la dernière mise-à-jour 
- numBikesAvailable: nombre de vélos disponibles
- numDocksAvailable: nombre de bornettes disponibles
- num_bikes_available: nombre de vélos disponibles
- num_bikes_available_types: nombre de vélos disponibles avec distinctions entre Vélib’ mécanique et électrique

In [18]:
request= urllib.request.Request('https://velib-metropole-opendata.smoove.pro/opendata/Velib_Metropole/station_status.json')
response = urllib.request.urlopen(request)
velib_stations = response.read()
data = json.loads(velib_stations)
df = json_normalize(data['data']['stations'])
df


,stationCode,station_id,num_bikes_available,numBikesAvailable,num_bikes_available_types,num_docks_available,numDocksAvailable,is_installed,is_returning,is_renting,last_reported
0,16107,213688169,3,3,"[{'mechanical': 1}, {'ebike': 2}]",30,30,1,1,1,1605558138
1,6015,99950133,14,14,"[{'mechanical': 11}, {'ebike': 3}]",41,41,1,1,1,1605558381
2,11104,516709288,9,9,"[{'mechanical': 6}, {'ebike': 3}]",11,11,1,1,1,1605558299
3,9020,36255,10,10,"[{'mechanical': 8}, {'ebike': 2}]",9,9,1,1,1,1605558511
4,12109,37815204,15,15,"[{'mechanical': 7}, {'ebike': 8}]",14,14,1,1,1,1605558498
...,...,...,...,...,...,...,...,...,...,...,...
1392,12039,27415004,20,20,"[{'mechanical': 14}, {'ebike': 6}]",16,16,1,1,1,1605558296
1393,13037,27415128,8,8,"[{'mechanical': 2}, {'ebike': 6}]",12,12,1,1,1,1605558367
1394,12017,27414937,34,34,"[{'mechanical': 21}, {'ebike': 13}]",5,5,1,1,1,1605558409
1395,13013,66507230,14,14,"[{'mechanical': 13}, {'ebike': 1}]",7,7,1,1,1,1605558488


Dans ce dataframe `df`, on peut rajouter 2 colonnes : mechanical & ebike au lieu de `num_bikes_available_types`

In [15]:
velib_clean = pd.read_csv('velib_clean.csv')
velib_clean.head()

,station_id,10/19/20;18:01:51,10/19/20;18:06:52,10/19/20;18:11:52,10/19/20;18:16:52,10/19/20;18:21:52,10/19/20;18:26:53,10/19/20;18:31:53,10/19/20;18:36:53,10/19/20;18:41:54,...,11/03/20;16:01:46,11/03/20;16:06:47,11/03/20;16:11:47,11/03/20;16:16:47,11/03/20;16:21:48,11/03/20;16:26:48,11/03/20;16:31:48,11/03/20;16:36:49,11/03/20;16:41:49,11/03/20;16:46:49
0,213688169,"{'mechanical': 0, 'ebike': 2}","{'mechanical': 0, 'ebike': 2}","{'mechanical': 0, 'ebike': 2}","{'mechanical': 0, 'ebike': 3}","{'mechanical': 0, 'ebike': 3}","{'mechanical': 1, 'ebike': 2}","{'mechanical': 0, 'ebike': 1}","{'mechanical': 0, 'ebike': 1}","{'mechanical': 0, 'ebike': 1}",...,"{'mechanical': 4, 'ebike': 6}","{'mechanical': 4, 'ebike': 6}","{'mechanical': 3, 'ebike': 6}","{'mechanical': 4, 'ebike': 7}","{'mechanical': 5, 'ebike': 7}","{'mechanical': 5, 'ebike': 7}","{'mechanical': 5, 'ebike': 8}","{'mechanical': 5, 'ebike': 8}","{'mechanical': 5, 'ebike': 8}","{'mechanical': 5, 'ebike': 7}"
1,99950133,"{'mechanical': 41, 'ebike': 5}","{'mechanical': 41, 'ebike': 5}","{'mechanical': 42, 'ebike': 5}","{'mechanical': 42, 'ebike': 7}","{'mechanical': 41, 'ebike': 5}","{'mechanical': 40, 'ebike': 4}","{'mechanical': 38, 'ebike': 5}","{'mechanical': 39, 'ebike': 4}","{'mechanical': 40, 'ebike': 5}",...,"{'mechanical': 2, 'ebike': 0}","{'mechanical': 2, 'ebike': 0}","{'mechanical': 2, 'ebike': 0}","{'mechanical': 2, 'ebike': 1}","{'mechanical': 2, 'ebike': 1}","{'mechanical': 2, 'ebike': 0}","{'mechanical': 2, 'ebike': 0}","{'mechanical': 2, 'ebike': 0}","{'mechanical': 2, 'ebike': 0}","{'mechanical': 2, 'ebike': 0}"
2,516709288,"{'mechanical': 0, 'ebike': 1}","{'mechanical': 0, 'ebike': 1}","{'mechanical': 0, 'ebike': 1}","{'mechanical': 1, 'ebike': 1}","{'mechanical': 1, 'ebike': 1}","{'mechanical': 1, 'ebike': 1}","{'mechanical': 2, 'ebike': 1}","{'mechanical': 0, 'ebike': 1}","{'mechanical': 0, 'ebike': 1}",...,"{'mechanical': 1, 'ebike': 2}","{'mechanical': 1, 'ebike': 2}","{'mechanical': 1, 'ebike': 2}","{'mechanical': 1, 'ebike': 2}","{'mechanical': 1, 'ebike': 2}","{'mechanical': 1, 'ebike': 3}","{'mechanical': 1, 'ebike': 2}","{'mechanical': 1, 'ebike': 3}","{'mechanical': 1, 'ebike': 3}","{'mechanical': 1, 'ebike': 3}"
3,36255,"{'mechanical': 0, 'ebike': 0}","{'mechanical': 0, 'ebike': 0}","{'mechanical': 0, 'ebike': 0}","{'mechanical': 0, 'ebike': 0}","{'mechanical': 0, 'ebike': 2}","{'mechanical': 0, 'ebike': 4}","{'mechanical': 0, 'ebike': 4}","{'mechanical': 0, 'ebike': 4}","{'mechanical': 1, 'ebike': 3}",...,"{'mechanical': 0, 'ebike': 3}","{'mechanical': 0, 'ebike': 3}","{'mechanical': 0, 'ebike': 3}","{'mechanical': 0, 'ebike': 3}","{'mechanical': 0, 'ebike': 2}","{'mechanical': 0, 'ebike': 2}","{'mechanical': 0, 'ebike': 3}","{'mechanical': 0, 'ebike': 3}","{'mechanical': 0, 'ebike': 3}","{'mechanical': 0, 'ebike': 2}"
4,37815204,"{'mechanical': 4, 'ebike': 1}","{'mechanical': 3, 'ebike': 1}","{'mechanical': 1, 'ebike': 1}","{'mechanical': 1, 'ebike': 1}","{'mechanical': 2, 'ebike': 1}","{'mechanical': 3, 'ebike': 1}","{'mechanical': 3, 'ebike': 2}","{'mechanical': 2, 'ebike': 0}","{'mechanical': 2, 'ebike': 0}",...,"{'mechanical': 6, 'ebike': 0}","{'mechanical': 7, 'ebike': 0}","{'mechanical': 7, 'ebike': 0}","{'mechanical': 7, 'ebike': 1}","{'mechanical': 7, 'ebike': 1}","{'mechanical': 8, 'ebike': 1}","{'mechanical': 9, 'ebike': 0}","{'mechanical': 10, 'ebike': 0}","{'mechanical': 9, 'ebike': 0}","{'mechanical': 9, 'ebike': 0}"
